In [2]:
import csv
import sys

In [3]:
# Maps names to a set of corresponding person_ids
names = {}

# Maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)
people = {}

# Maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}

In [4]:
class Node():
    def __init__(self, state, parent):  #, action):
        self.state = state
        self.parent = parent
#        self.action = action

In [5]:
class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node

In [6]:
class QueueFrontier(StackFrontier):

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node

In [7]:
def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"{directory}/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"{directory}/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"{directory}/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass

In [8]:
def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]

In [9]:
def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people
    who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors

In [10]:
directory = 'small'

In [11]:
print("Loading data...")
load_data(directory)
print("Data loaded.")

Loading data...
Data loaded.


In [193]:
source = person_id_for_name('cary elwes')
print(source)

144


In [194]:
target = person_id_for_name('demi moore')
print(target)

193


In [195]:
movie_ids = people[source]["movies"]

In [196]:
frontier = StackFrontier()

In [197]:
explored = set()

In [198]:
# load starting nodes
for movie in movie_ids:
    start_state = (movie, source)
    start_node = Node(state=start_state, parent=None)
    frontier.add(start_node)

Node: ('109830', '705')


In [199]:
while True:

    # If nothing left in frontier, then no path
    if frontier.empty():
        raise Exception("no solution")

    # Choose a node from the frontier
    node = frontier.remove()
    num_explored += 1
    
    # check for a match
    if node.state[1] == target:
        print("match found")
        print(node.state)
        solution = []
        while node.parent is not None:
            solution.append(node.state)
            node = node.parent
#        solution.append(node.state)
#        solution.reverse
        break

    # Mark node as explored
    explored.add(node.state)

    # Add neighbors to frontier
    for state in neighbors_for_person(node.state[1]):
        if not frontier.contains_state(state) and state not in explored:
            child = Node(state=state, parent=node)
            frontier.add(child)


match found
('104257', '193')


In [200]:
if solution is None:
    print("Not connected.")
else:
    degrees = len(solution)
    print(f"{degrees} degrees of separation.")
    solution.append((None, source))
    for i in range(degrees):
        person1 = people[solution[i][1]]["name"]
        person2 = people[solution[i + 1][1]]["name"]
        movie = movies[solution[i][0]]["title"]
        print(f"{i + 1}: {person1} and {person2} starred in {movie}")

4 degrees of separation.
1: Demi Moore and Kevin Bacon starred in A Few Good Men
2: Kevin Bacon and Tom Hanks starred in Apollo 13
3: Tom Hanks and Robin Wright starred in Forrest Gump
4: Robin Wright and Cary Elwes starred in The Princess Bride
